In [20]:
import pandas as pd
from dateutil.relativedelta import relativedelta
from common import Data

In [3]:
stats = Data.get_nwsl_player_stats()
stats

<ipython-input-3-6677b5ba1082>:1: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  stats = Data.get_nwsl_player_stats()


,season,date,home,away,player,team,number,position,is_starter,accurate_keeper_throws,...,dive_catch,att_lg_centre,foul_throw_in,assist_handball_won,att_pen_post,att_obp_goal,att_pen_miss,assist_post,att_lg_left,back_pass
0,2016,2016-04-16,HOU,CHI,Lydia Grace Yilkari Williams,HOU,18,Goalkeeper,True,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2016,2016-04-16,HOU,CHI,Poliana,HOU,2,Defender,True,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2016,2016-04-16,HOU,CHI,Rebecca Moros,HOU,4,Defender,True,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2016,2016-04-16,HOU,CHI,Allysha Chapman,HOU,15,Defender,True,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2016,2016-04-16,HOU,CHI,Ellie Katherine Brush,HOU,24,Defender,True,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36883,2023 Challenge Cup,2023-09-09,NC,LOU,Parker Goins,LOU,21,Substitute,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36884,2023 Challenge Cup,2023-09-09,NC,LOU,Uchenna Kanu,LOU,29,Substitute,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36885,2023 Challenge Cup,2023-09-09,NC,LOU,Nadia Nadim,LOU,10,Substitute,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36886,2023 Challenge Cup,2023-09-09,NC,LOU,Maddie Pokorny,LOU,17,Substitute,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
players = Data.get_nwsl_players()
players

,name,position,birthday,country,hometown,number,height
0,Jane Campbell,Goalkeeper,1995-02-17,USA,Kennesaw,1,"5'8"""
1,Katie Lund,Goalkeeper,1996-11-27,USA,Plano,1,"6'0"""
2,Alyssa Naeher,Goalkeeper,1988-04-20,USA,Bridgeport,1,"5'8"""
3,Anna Moorhouse,Goalkeeper,1995-03-30,England,Oldham,21,NaN
4,Bella Bixby,Goalkeeper,1995-11-20,USA,Milwaukie,1,"6'0"""
...,...,...,...,...,...,...,...
727,Jami-Ann Kranich,Goalkeeper,1992-05-27,USA,Hamden,2,"5'10"""
728,Hope Solo,Goalkeeper,1981-07-30,USA,Richland,1,"5'8"""
729,Kaitlyn Savage,Goalkeeper,1990-10-27,USA,NaN,25,"5'10"""
730,Bianca Henninger,Goalkeeper,1990-10-22,USA,Los Gatos,1,"5'6"""


In [22]:
df = stats.set_index("player")[["season", "date", "team", "mins_played"]].join(
    players.set_index("name")[["birthday"]]
)
df = df.dropna()
df = df.reset_index().rename(columns={"index": "player"})
df["date"] = pd.to_datetime(df["date"])
df["birthday"] = pd.to_datetime(df["birthday"])
df

,player,season,date,team,mins_played,birthday
0,Abby Dahlkemper,2016,2016-04-16,WNY,90.0,1993-05-13
1,Abby Dahlkemper,2016,2016-04-23,WNY,90.0,1993-05-13
2,Abby Dahlkemper,2016,2016-04-29,WNY,90.0,1993-05-13
3,Abby Dahlkemper,2016,2016-05-07,WNY,90.0,1993-05-13
4,Abby Dahlkemper,2016,2016-05-14,WNY,90.0,1993-05-13
...,...,...,...,...,...,...
26806,Évelyne Viens,2021,2021-05-15,NJY,69.0,1997-02-06
26807,Évelyne Viens,2021,2021-05-30,NJY,14.0,1997-02-06
26808,Évelyne Viens,2021 Challenge Cup,2021-04-20,NJY,22.0,1997-02-06
26809,Évelyne Viens,2021 Challenge Cup,2021-05-02,NJY,1.0,1997-02-06


In [18]:
dob = df.groupby(["player", "season", "birthday"]).agg(
    {"mins_played": "sum"}
).reset_index().rename(
    columns={
        "player": "name",
        "season": "year",
        "mins_played": "minutes",
        "birthday": "dob",
    }
)

dob.to_csv("data/dob_and_minutes.csv", index=False)
dob

,name,year,dob,minutes
0,Abby Dahlkemper,2016,1993-05-13,1920.0
1,Abby Dahlkemper,2017,1993-05-13,2160.0
2,Abby Dahlkemper,2018,1993-05-13,1890.0
3,Abby Dahlkemper,2019,1993-05-13,1470.0
4,Abby Dahlkemper,2020 Challenge Cup,1993-05-13,270.0
...,...,...,...,...
2693,Zoe Morse,2022,1998-04-01,1829.0
2694,Zoey Goralski,2019,1995-01-22,59.0
2695,Évelyne Viens,2020 Challenge Cup,1997-02-06,114.0
2696,Évelyne Viens,2021,1997-02-06,241.0


In [24]:
df["age"] = df.apply(lambda x: relativedelta(x["date"], x["birthday"]).years, axis=1)
df

,player,season,date,team,mins_played,birthday,age
0,Abby Dahlkemper,2016,2016-04-16,WNY,90.0,1993-05-13,22
1,Abby Dahlkemper,2016,2016-04-23,WNY,90.0,1993-05-13,22
2,Abby Dahlkemper,2016,2016-04-29,WNY,90.0,1993-05-13,22
3,Abby Dahlkemper,2016,2016-05-07,WNY,90.0,1993-05-13,22
4,Abby Dahlkemper,2016,2016-05-14,WNY,90.0,1993-05-13,23
...,...,...,...,...,...,...,...
26806,Évelyne Viens,2021,2021-05-15,NJY,69.0,1997-02-06,24
26807,Évelyne Viens,2021,2021-05-30,NJY,14.0,1997-02-06,24
26808,Évelyne Viens,2021 Challenge Cup,2021-04-20,NJY,22.0,1997-02-06,24
26809,Évelyne Viens,2021 Challenge Cup,2021-05-02,NJY,1.0,1997-02-06,24


In [31]:
minutes = (
    df.rename(columns={"season": "year"})
    .groupby(["team", "year", "age"])
    .agg({"mins_played": "sum"})
    .reset_index()
    .pivot(index=["team", "year"], columns="age", values="mins_played")
)

minutes.to_csv("data/nwsl_minutes.csv")
minutes

age                        15     16  17  18     19      20      21      22  \
team year                                                                     
BOS  2016                 NaN    NaN NaN NaN    NaN     NaN   100.0  1464.0   
     2017                 NaN    NaN NaN NaN    NaN     NaN  1559.0  1626.0   
CHI  2016                 NaN    NaN NaN NaN    NaN     NaN     NaN  1654.0   
     2017                 NaN    NaN NaN NaN    NaN     NaN   134.0   327.0   
     2018                 NaN    NaN NaN NaN    NaN     NaN    90.0   521.0   
...                       ...    ...  ..  ..    ...     ...     ...     ...   
WAS  2022                 NaN    NaN NaN NaN  270.0  1234.0     NaN   300.0   
     2022 Challenge Cup   NaN    NaN NaN NaN  702.0     NaN     NaN   418.0   
     2023                75.0  213.0 NaN NaN    NaN   624.0  1011.0  1731.0   
     2023 Challenge Cup  74.0  251.0 NaN NaN    NaN     NaN     NaN   659.0   
WNY  2016                 NaN    NaN NaN NaN    NaN    45.0  1508.0  4708.0   

age                          23      24  ...     33     34  35  36  37  38  \
team year                                ...                                 
BOS  2016                3273.0  4420.0  ...    NaN    NaN NaN NaN NaN NaN   
     2017                4395.0  5899.0  ...  804.0    NaN NaN NaN NaN NaN   
CHI  2016                7350.0  3332.0  ...    NaN    NaN NaN NaN NaN NaN   
     2017                2772.0  6825.0  ...    NaN    NaN NaN NaN NaN NaN   
     2018                 485.0  6049.0  ...    NaN    NaN NaN NaN NaN NaN   
...                         ...     ...  ...    ...    ...  ..  ..  ..  ..   
WAS  2022                6404.0   708.0  ...  195.0  180.0 NaN NaN NaN NaN   
     2022 Challenge Cup  2134.0   403.0  ...  574.0    NaN NaN NaN NaN NaN   
     2023                1951.0  4059.0  ...   19.0    NaN NaN NaN NaN NaN   
     2023 Challenge Cup  1131.0  1035.0  ...   36.0    NaN NaN NaN NaN NaN   
WNY  2016                7045.0  2241.0  ...    NaN    NaN NaN NaN NaN NaN   

age                      39  40     41  42  
team year                                   
BOS  2016               NaN NaN    NaN NaN  
     2017               NaN NaN    NaN NaN  
CHI  2016               NaN NaN    NaN NaN  
     2017               NaN NaN    NaN NaN  
     2018               NaN NaN    NaN NaN  
...                      ..  ..    ...  ..  
WAS  2022               NaN NaN    NaN NaN  
     2022 Challenge Cup NaN NaN    NaN NaN  
     2023               NaN NaN  181.0 NaN  
     2023 Challenge Cup NaN NaN  360.0 NaN  
WNY  2016               NaN NaN    NaN NaN  

[118 rows x 28 columns]